In [1]:
!pip install xgboost

In [2]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import mean_squared_error
import itertools

In [3]:
X_train = pd.read_csv("Training_Validation_Test_Datasets/task2_input_train.csv")
X_validate = pd.read_csv("Training_Validation_Test_Datasets/task2_input_validate.csv")
X_test = pd.read_csv("Training_Validation_Test_Datasets/task2_input_test.csv")

y_train = pd.read_csv("Training_Validation_Test_Datasets/task2_output_train.csv")["phq_sum"]
y_validate = pd.read_csv("Training_Validation_Test_Datasets/task2_output_validate.csv")["phq_sum"]
y_test = pd.read_csv("Training_Validation_Test_Datasets/task2_output_test.csv")["phq_sum"]

In [4]:
X_train

,com01,com02,com03,com07,com08,com09,com10,com13,com15,com16,...,lab10,lab11,lab12,lab14,physical01,physical02,physical03,physical04,physical05,physical06
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,...,19.64,0.735,1.40,1.57,63.7,162.5,24.1,96.0,58.0,57.0
1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,...,2.90,1.209,0.26,1.57,90.5,150.5,40.0,153.0,91.0,52.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,...,0.55,2.301,2.01,1.57,87.7,179.8,27.1,114.0,61.0,60.0
3,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,...,0.64,0.277,0.21,1.57,72.1,167.9,25.6,138.0,86.0,71.0
4,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,2.0,...,3.91,1.313,0.46,1.57,67.2,157.2,27.2,109.0,75.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,...,3.57,1.202,1.33,1.57,65.6,156.2,26.9,147.0,95.0,79.0
3307,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,...,1.49,1.160,1.72,1.57,101.2,173.3,33.7,150.0,88.0,53.0
3308,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,...,1.53,1.037,0.94,1.57,113.0,186.1,32.6,121.0,85.0,76.0
3309,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,...,0.53,0.482,1.04,1.57,57.7,162.5,21.9,109.0,70.0,58.0


In [5]:
# Convert to DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalidate = xgb.DMatrix(X_validate, label=y_validate)
dtest = xgb.DMatrix(X_test, label=y_test)

In [6]:
evals = [(dtrain, "train"), (dvalidate, "validate")]

# Hyperparameter Tuning
depths = [1,2,3,4,5,6,7]
etas = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
subsample = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
colsample_bytree = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

# Get all combinations
combinations = list(itertools.product(depths, etas, subsample, colsample_bytree))

In [7]:
best_hyperparameter = None
best_rmse = None

In [8]:
for i in combinations:
    print(f"Hyperparameters: max_depth({i[0]}), eta({i[1]}), subsample({i[2]}), colsample_bytree({i[3]})")
    
    params = {
    "objective": "reg:squarederror",  
    "eval_metric": "rmse",            
    "seed": 42                      
}

    params["max_depth"] = i[0]
    params["eta"] = i[1]
    params["subsample"] = i[2]
    params["colsample_bytree"] = i[3]

    model = xgb.train(params, dtrain, num_boost_round=500,
                      evals=evals, early_stopping_rounds=50, verbose_eval=100)
    
    y_pred = model.predict(dvalidate)
    rmse = mean_squared_error(y_validate, y_pred)**(1/2)

    if best_rmse is None or rmse < best_rmse:
        best_rmse = rmse
        best_hyperparameter = {
            "max_depth": i[0],
            "eta": i[1],
            "subsample": i[2],
            "colsample_bytree": i[3]
        }

Hyperparameters: max_depth(1), eta(0.01), subsample(0.1), colsample_bytree(0.1)
[0]	train-rmse:4.77344	validate-rmse:4.76915
[100]	train-rmse:4.55716	validate-rmse:4.54016
[200]	train-rmse:4.39219	validate-rmse:4.38098
[300]	train-rmse:4.27165	validate-rmse:4.26296
[400]	train-rmse:4.16313	validate-rmse:4.15878
[499]	train-rmse:4.06919	validate-rmse:4.06395
Hyperparameters: max_depth(1), eta(0.01), subsample(0.1), colsample_bytree(0.2)
[0]	train-rmse:4.77344	validate-rmse:4.76915
[100]	train-rmse:4.48755	validate-rmse:4.46878
[200]	train-rmse:4.28760	validate-rmse:4.26843
[300]	train-rmse:4.14608	validate-rmse:4.13174
[400]	train-rmse:4.03728	validate-rmse:4.03098
[499]	train-rmse:3.95775	validate-rmse:3.96091
Hyperparameters: max_depth(1), eta(0.01), subsample(0.1), colsample_bytree(0.3)
[0]	train-rmse:4.77344	validate-rmse:4.76915
[100]	train-rmse:4.44164	validate-rmse:4.42557
[200]	train-rmse:4.23207	validate-rmse:4.22108
[300]	train-rmse:4.10011	validate-rmse:4.09686
[400]	train-rm

[499]	train-rmse:3.96771	validate-rmse:3.98157
Hyperparameters: max_depth(1), eta(0.01), subsample(0.4), colsample_bytree(0.3)
[0]	train-rmse:4.77414	validate-rmse:4.77000
[100]	train-rmse:4.44188	validate-rmse:4.42304
[200]	train-rmse:4.23658	validate-rmse:4.22348
[300]	train-rmse:4.10295	validate-rmse:4.09582
[400]	train-rmse:4.01087	validate-rmse:4.01252
[499]	train-rmse:3.94418	validate-rmse:3.95990
Hyperparameters: max_depth(1), eta(0.01), subsample(0.4), colsample_bytree(0.4)
[0]	train-rmse:4.77414	validate-rmse:4.77000
[100]	train-rmse:4.42245	validate-rmse:4.40181
[200]	train-rmse:4.21501	validate-rmse:4.19445
[300]	train-rmse:4.08780	validate-rmse:4.07684
[400]	train-rmse:3.99759	validate-rmse:3.99774
[499]	train-rmse:3.93417	validate-rmse:3.94471
Hyperparameters: max_depth(1), eta(0.01), subsample(0.4), colsample_bytree(0.5)
[0]	train-rmse:4.77414	validate-rmse:4.77000
[100]	train-rmse:4.41125	validate-rmse:4.39797
[200]	train-rmse:4.20507	validate-rmse:4.18812
[300]	train-rm

[400]	train-rmse:4.00981	validate-rmse:4.01612
[499]	train-rmse:3.94856	validate-rmse:3.96539
Hyperparameters: max_depth(1), eta(0.01), subsample(0.7), colsample_bytree(0.5)
[0]	train-rmse:4.77464	validate-rmse:4.77062
[100]	train-rmse:4.41614	validate-rmse:4.40483
[200]	train-rmse:4.21333	validate-rmse:4.19726
[300]	train-rmse:4.09087	validate-rmse:4.08535
[400]	train-rmse:4.00502	validate-rmse:4.01366
[499]	train-rmse:3.94341	validate-rmse:3.96579
Hyperparameters: max_depth(1), eta(0.01), subsample(0.7), colsample_bytree(0.6)
[0]	train-rmse:4.77464	validate-rmse:4.77062
[100]	train-rmse:4.41424	validate-rmse:4.40516
[200]	train-rmse:4.20986	validate-rmse:4.19311
[300]	train-rmse:4.08863	validate-rmse:4.08230
[400]	train-rmse:4.00329	validate-rmse:4.01158
[499]	train-rmse:3.94190	validate-rmse:3.96224
Hyperparameters: max_depth(1), eta(0.01), subsample(0.7), colsample_bytree(0.7)
[0]	train-rmse:4.77464	validate-rmse:4.77062
[100]	train-rmse:4.41113	validate-rmse:4.40105
[200]	train-rm

[300]	train-rmse:3.86900	validate-rmse:3.89116
[400]	train-rmse:3.80712	validate-rmse:3.84500
[499]	train-rmse:3.77035	validate-rmse:3.83221
Hyperparameters: max_depth(1), eta(0.02), subsample(0.3), colsample_bytree(0.7)
[0]	train-rmse:4.76952	validate-rmse:4.76426
[100]	train-rmse:4.19231	validate-rmse:4.18014
[200]	train-rmse:3.97748	validate-rmse:3.97852
[300]	train-rmse:3.86781	validate-rmse:3.89538
[400]	train-rmse:3.80632	validate-rmse:3.85331
[499]	train-rmse:3.76810	validate-rmse:3.83502
Hyperparameters: max_depth(1), eta(0.02), subsample(0.4), colsample_bytree(0.1)
[0]	train-rmse:4.76949	validate-rmse:4.76421
[100]	train-rmse:4.38318	validate-rmse:4.36922
[200]	train-rmse:4.15517	validate-rmse:4.14878
[300]	train-rmse:4.00870	validate-rmse:4.01243
[400]	train-rmse:3.90931	validate-rmse:3.93913
[499]	train-rmse:3.84799	validate-rmse:3.88807
Hyperparameters: max_depth(1), eta(0.02), subsample(0.4), colsample_bytree(0.2)
[0]	train-rmse:4.76949	validate-rmse:4.76421
[100]	train-rm

[200]	train-rmse:4.15956	validate-rmse:4.15611
[300]	train-rmse:4.01819	validate-rmse:4.02651
[400]	train-rmse:3.91669	validate-rmse:3.94368
[499]	train-rmse:3.85338	validate-rmse:3.89223
Hyperparameters: max_depth(1), eta(0.02), subsample(0.7), colsample_bytree(0.2)
[0]	train-rmse:4.77048	validate-rmse:4.76544
[100]	train-rmse:4.29021	validate-rmse:4.28040
[200]	train-rmse:4.04812	validate-rmse:4.05351
[300]	train-rmse:3.92665	validate-rmse:3.95025
[400]	train-rmse:3.85143	validate-rmse:3.89414
[499]	train-rmse:3.80727	validate-rmse:3.86140
Hyperparameters: max_depth(1), eta(0.02), subsample(0.7), colsample_bytree(0.3)
[0]	train-rmse:4.77048	validate-rmse:4.76544
[100]	train-rmse:4.23926	validate-rmse:4.22604
[200]	train-rmse:4.01573	validate-rmse:4.02039
[300]	train-rmse:3.90501	validate-rmse:3.93020
[400]	train-rmse:3.83980	validate-rmse:3.87960
[499]	train-rmse:3.80003	validate-rmse:3.85389
Hyperparameters: max_depth(1), eta(0.02), subsample(0.7), colsample_bytree(0.4)
[0]	train-rm

...

[100]	train-rmse:2.88245	validate-rmse:3.90914
[143]	train-rmse:2.62490	validate-rmse:3.92518
Hyperparameters: max_depth(7), eta(0.04), subsample(0.2), colsample_bytree(0.7)
[0]	train-rmse:4.73042	validate-rmse:4.74003
[100]	train-rmse:2.88906	validate-rmse:3.91305
[159]	train-rmse:2.53615	validate-rmse:3.93153
Hyperparameters: max_depth(7), eta(0.04), subsample(0.3), colsample_bytree(0.1)
[0]	train-rmse:4.73526	validate-rmse:4.74122
[100]	train-rmse:3.29154	validate-rmse:4.03664
[200]	train-rmse:2.67079	validate-rmse:3.93036
[300]	train-rmse:2.25673	validate-rmse:3.90876
[337]	train-rmse:2.13123	validate-rmse:3.90930
Hyperparameters: max_depth(7), eta(0.04), subsample(0.3), colsample_bytree(0.2)
[0]	train-rmse:4.73316	validate-rmse:4.74081
[100]	train-rmse:2.93052	validate-rmse:3.96238
[199]	train-rmse:2.33436	validate-rmse:3.93453
Hyperparameters: max_depth(7), eta(0.04), subsample(0.3), colsample_bytree(0.3)
[0]	train-rmse:4.73024	validate-rmse:4.74227
[100]	train-rmse:2.81111	valid

[100]	train-rmse:2.26059	validate-rmse:3.85482
[166]	train-rmse:1.76127	validate-rmse:3.85545
Hyperparameters: max_depth(7), eta(0.04), subsample(0.7), colsample_bytree(0.6)
[0]	train-rmse:4.70946	validate-rmse:4.73014
[100]	train-rmse:2.22203	validate-rmse:3.87664
[136]	train-rmse:1.94720	validate-rmse:3.88086
Hyperparameters: max_depth(7), eta(0.04), subsample(0.7), colsample_bytree(0.7)
[0]	train-rmse:4.70602	validate-rmse:4.72338
[100]	train-rmse:2.22722	validate-rmse:3.88846
[119]	train-rmse:2.06596	validate-rmse:3.88256
Hyperparameters: max_depth(7), eta(0.05), subsample(0.1), colsample_bytree(0.1)
[0]	train-rmse:4.72828	validate-rmse:4.73371
[100]	train-rmse:3.53820	validate-rmse:4.07012
[148]	train-rmse:3.31787	validate-rmse:4.07162
Hyperparameters: max_depth(7), eta(0.05), subsample(0.1), colsample_bytree(0.2)
[0]	train-rmse:4.72390	validate-rmse:4.72790
[100]	train-rmse:3.34600	validate-rmse:3.99673
[148]	train-rmse:3.11921	validate-rmse:4.01612
Hyperparameters: max_depth(7),

Hyperparameters: max_depth(7), eta(0.05), subsample(0.5), colsample_bytree(0.7)
[0]	train-rmse:4.69596	validate-rmse:4.71757
[100]	train-rmse:2.21340	validate-rmse:3.89064
[107]	train-rmse:2.14893	validate-rmse:3.89330
Hyperparameters: max_depth(7), eta(0.05), subsample(0.6), colsample_bytree(0.1)
[0]	train-rmse:4.71924	validate-rmse:4.72357
[100]	train-rmse:2.83157	validate-rmse:3.95436
[200]	train-rmse:2.11750	validate-rmse:3.89293
[268]	train-rmse:1.80720	validate-rmse:3.88667
Hyperparameters: max_depth(7), eta(0.05), subsample(0.6), colsample_bytree(0.2)
[0]	train-rmse:4.70996	validate-rmse:4.72898
[100]	train-rmse:2.44168	validate-rmse:3.85531
[200]	train-rmse:1.71358	validate-rmse:3.83341
[240]	train-rmse:1.52071	validate-rmse:3.83394
Hyperparameters: max_depth(7), eta(0.05), subsample(0.6), colsample_bytree(0.3)
[0]	train-rmse:4.70264	validate-rmse:4.72925
[100]	train-rmse:2.27812	validate-rmse:3.85107
[152]	train-rmse:1.87619	validate-rmse:3.86300
Hyperparameters: max_depth(7),

[200]	train-rmse:2.13835	validate-rmse:3.91354
[232]	train-rmse:1.98787	validate-rmse:3.91936
Hyperparameters: max_depth(7), eta(0.06), subsample(0.4), colsample_bytree(0.2)
[0]	train-rmse:4.69939	validate-rmse:4.71346
[100]	train-rmse:2.46842	validate-rmse:3.86972
[170]	train-rmse:1.93008	validate-rmse:3.86830
Hyperparameters: max_depth(7), eta(0.06), subsample(0.4), colsample_bytree(0.3)
[0]	train-rmse:4.69981	validate-rmse:4.70573
[100]	train-rmse:2.31646	validate-rmse:3.95704
[156]	train-rmse:1.86932	validate-rmse:3.97364
Hyperparameters: max_depth(7), eta(0.06), subsample(0.4), colsample_bytree(0.4)
[0]	train-rmse:4.69109	validate-rmse:4.70232
[100]	train-rmse:2.23819	validate-rmse:3.94138
[113]	train-rmse:2.12464	validate-rmse:3.94257
Hyperparameters: max_depth(7), eta(0.06), subsample(0.4), colsample_bytree(0.5)
[0]	train-rmse:4.68204	validate-rmse:4.71626
[98]	train-rmse:2.25375	validate-rmse:3.98378
Hyperparameters: max_depth(7), eta(0.06), subsample(0.4), colsample_bytree(0.6

[96]	train-rmse:2.61005	validate-rmse:4.05023
Hyperparameters: max_depth(7), eta(0.07), subsample(0.2), colsample_bytree(0.6)
[0]	train-rmse:4.69961	validate-rmse:4.70822
[78]	train-rmse:2.75785	validate-rmse:4.04235
Hyperparameters: max_depth(7), eta(0.07), subsample(0.2), colsample_bytree(0.7)
[0]	train-rmse:4.69639	validate-rmse:4.71601
[97]	train-rmse:2.54726	validate-rmse:3.97712
Hyperparameters: max_depth(7), eta(0.07), subsample(0.3), colsample_bytree(0.1)
[0]	train-rmse:4.70400	validate-rmse:4.71704
[100]	train-rmse:2.84968	validate-rmse:4.01876
[200]	train-rmse:2.18013	validate-rmse:4.01142
[224]	train-rmse:2.06328	validate-rmse:4.01216
Hyperparameters: max_depth(7), eta(0.07), subsample(0.3), colsample_bytree(0.2)
[0]	train-rmse:4.70048	validate-rmse:4.71625
[100]	train-rmse:2.48757	validate-rmse:3.94007
[130]	train-rmse:2.24802	validate-rmse:3.96032
Hyperparameters: max_depth(7), eta(0.07), subsample(0.3), colsample_bytree(0.3)
[0]	train-rmse:4.69539	validate-rmse:4.71886
[1

In [9]:
print(f"Best Hyperparameters: {best_hyperparameter}")
print("RMSE: ", best_rmse)

Best Hyperparameters: {'max_depth': 3, 'eta': 0.06, 'subsample': 0.6, 'colsample_bytree': 0.2}
RMSE:  3.786385054850448


In [10]:
params = {
        "objective": "reg:squarederror",  
        "eval_metric": "rmse",      
        "eta": 0.06,                     
        "max_depth": 3,      
        "subsample": 0.6,       
        "colsample_bytree": 0.2,
    }

In [11]:
# Use the best hyperparameters to produce output
model = xgb.train(params, dtrain, num_boost_round=500, evals=evals, 
                  early_stopping_rounds=50, verbose_eval=100)

y_pred = model.predict(dtest)
rmse = mean_squared_error(y_test, y_pred)**(1/2)

print("RMSE: ", rmse)

[0]	train-rmse:4.75329	validate-rmse:4.74858
[100]	train-rmse:3.56083	validate-rmse:3.82355
[154]	train-rmse:3.41495	validate-rmse:3.83113
RMSE:  3.7937705835685365
